# Introduction: Automated Feature Engineering

In this notebook, we will look at an exciting development in data science: automated feature engineering. A machine learning model can only learn from the data we give it, and making sure that data is relevant to the task is one of the most crucial steps in the machine learning pipeline (this is made clear in the excellent paper ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)). 

However, manual feature engineering is a tedious task and is limited by both human imagination - there are only so many features we can think to create - and by time - creating new features is time-intensive. Ideally, there would be an objective method to create an array of diverse new candidate features that we can then use for a machine learning task. This process is meant to not replace the data scientist, but to make her job easier and allowing her to supplement domain knowledge with an automated workflow.

In this notebook, we will walk through an implementation of using [Featuretools](https://www.featuretools.com/), an open-source Python library for automatically creating features with relational data (where the data is in structured tables). Although there are now many efforts working to enable automated model selection and hyperparameter tuning, there has been a lack of automating work on the feature engineering aspect of the pipeline. This library seeks to close that gap and the general methodology has been proven effective in both [machine learning competitions with the data science machine](https://github.com/HDI-Project/Data-Science-Machine) and [business use cases](https://www.featurelabs.com/blog/predicting-credit-card-fraud/). 


## Dataset

To show the basic idea of featuretools we will use an example dataset consisting of three tables:

* `clients`: information about clients at a credit union
* `loans`: previous loans taken out by the clients
* `payments`: payments made/missed on the previous loans

The general problem of feature engineering is taking disparate data, often distributed across multiple tables, and combining it into a single table that can be used for training a machine learning model. Featuretools has the ability to do this for us, creating many new candidate features with minimal effort. These features are combined into a single table that can then be passed on to our model. 

First, let's load in the data and look at the problem we are working with.

In [1]:
# Run this if featuretools is not already installed
# !pip install featuretools
# !pip install dask
# !pip install "dask[dataframe]" --upgrade

In [2]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# featuretools for automated feature engineering
import featuretools as ft

# ignore warnings from pandas
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Read in the data
clients = pd.read_csv('data/clients.csv', parse_dates = ['joined'])
loans = pd.read_csv('data/loans.csv', parse_dates = ['loan_start', 'loan_end'])
payments = pd.read_csv('data/payments.csv', parse_dates = ['payment_date'])

In [4]:
clients.head(5)

,client_id,joined,income,credit_score
0,46109,2002-04-16,172677,527
1,49545,2007-11-14,104564,770
2,41480,2013-03-11,122607,585
3,46180,2001-11-06,43851,562
4,25707,2006-10-06,211422,621


In [5]:
loans.head(5)

,client_id,loan_type,loan_amount,repaid,loan_id,loan_start,loan_end,rate
0,46109,home,13672,0,10243,2002-04-16,2003-12-20,2.15
1,46109,credit,9794,0,10984,2003-10-21,2005-07-17,1.25
2,46109,home,12734,1,10990,2006-02-01,2007-07-05,0.68
3,46109,cash,12518,1,10596,2010-12-08,2013-05-05,1.24
4,46109,credit,14049,1,11415,2010-07-07,2012-05-21,3.13


In [6]:
payments.head(5)

,loan_id,payment_amount,payment_date,missed
0,10243,2369,2002-05-31,1
1,10243,2439,2002-06-18,1
2,10243,2662,2002-06-29,0
3,10243,2268,2002-07-20,0
4,10243,2027,2002-07-31,1


### Manual Feature Engineering Examples

Let's show a few examples of features we might make by hand. We will keep this relatively simple to avoid doing too much work! First we will focus on a single dataframe before combining them together. In the `clients` dataframe, we can take the month of the `joined` column and the natural log of the `income` column. Later, we see these are known in featuretools as transformation feature primitives because they act on column in a single table. 

In [7]:
clients.head(2)

,client_id,joined,income,credit_score
0,46109,2002-04-16,172677,527
1,49545,2007-11-14,104564,770


### Data Transformation example

In [8]:
# Create a month column
clients['join_month'] = clients['joined'].dt.month
# Create a log of income column
clients['log_income'] = np.log(clients['income'])
clients.head()

,client_id,joined,income,credit_score,join_month,log_income
0,46109,2002-04-16,172677,527,4,12.059178
1,49545,2007-11-14,104564,770,11,11.557555
2,41480,2013-03-11,122607,585,3,11.716739
3,46180,2001-11-06,43851,562,11,10.688553
4,25707,2006-10-06,211422,621,10,12.261611


To incorporate information about the other tables, we use the `df.groupby` method, followed by a suitable aggregation function, followed by `df.merge`.  For example, let's calculate the average, minimum, and maximum amount of previous loans for each client. In the terms of featuretools, this would be considered an aggregation feature primitive because we using multiple tables in a one-to-many relationship to calculate aggregation figures (don't worry, this will be explained shortly!).

### Data aggregation example (between loans table and clients table)

In [9]:
# Groupby client id and calculate mean, max, min previous loan size
stats = loans.groupby('client_id')['loan_amount'].agg(['mean', 'max', 'min'])
stats.columns = ['mean_loan_amount', 'max_loan_amount', 'min_loan_amount']
stats.head()

,mean_loan_amount,max_loan_amount,min_loan_amount
client_id,,,
25707,7963.950000,13913,1212
26326,7270.062500,13464,1164
26695,7824.722222,14865,2389
26945,7125.933333,14593,653
29841,9813.000000,14837,2778


In [10]:
# Merge with the clients dataframe
clients.merge(stats, left_on = 'client_id', right_index=True, how = 'left').head(5)

,client_id,joined,income,credit_score,join_month,log_income,mean_loan_amount,max_loan_amount,min_loan_amount
0,46109,2002-04-16,172677,527,4,12.059178,8951.60,14049,559
1,49545,2007-11-14,104564,770,11,11.557555,10289.30,14971,3851
2,41480,2013-03-11,122607,585,3,11.716739,7894.85,14399,811
3,46180,2001-11-06,43851,562,11,10.688553,7700.85,14081,1607
4,25707,2006-10-06,211422,621,10,12.261611,7963.95,13913,1212


We could go further and include information about `payments` in the `clients` dataframe. To do so, we would have to group `payments` by the `loan_id`, merge it with the `loans`, group the resulting dataframe by the `client_id`, and then merge it into the `clients` dataframe. This would allow us to include information about previous payments for each client. 

Clearly, this process of manual feature engineering can grow quite tedious with many columns and multiple tables and I certainly don't want to have to do this process by hand! Luckily, featuretools can automatically perform this entire process and will create more features than we would have ever thought of. Although I love `pandas`, there is only so much manual data manipulation I'm willing to stand! 

# Featuretools

Now that we know what we are trying to avoid (tedious manual feature engineering), let's figure out how to automate this process. Featuretools operates on an idea known as [Deep Feature Synthesis](https://docs.featuretools.com/api_reference.html#deep-feature-synthesis). You can read the [original paper here](http://www.jmaxkanter.com/static/papers/DSAA_DSM_2015.pdf), and although it's quite readable, it's not necessary to understand the details to do automated feature engineering. The concept of Deep Feature Synthesis is to use basic building blocks known as feature primitives (like the transformations and aggregations done above) that can be stacked on top of each other to form new features. The depth of a "deep feature" is equal to the number of stacked primitives. 

I threw out some terms there, but don't worry because we'll cover them as we go. Featuretools builds on simple ideas to create a powerful method, and we will build up our understanding in much the same way. 

The first part of Featuretools to understand [is an `entity`](https://docs.featuretools.com/loading_data/using_entitysets.html#adding-entities). This is simply a table, or in `pandas`, a `DataFrame`. We corral multiple entities into a [single object called an `EntitySet`](https://docs.featuretools.com/loading_data/using_entitysets.html). This is just a large data structure composed of many individual entities and the relationships between them.  

## EntitySet

Creating a new `EntitySet` is pretty simple: 

In [11]:
es = ft.EntitySet(id = 'clients')
es

Entityset: clients
  Entities:
  Relationships:
    No relationships

## Entities 

An entity is simply a table, which is represented in Pandas as a `dataframe`. Each entity must have a uniquely identifying column, known as an index. For the clients dataframe, this is the `client_id` because each id only appears once in the `clients` data. In the `loans` dataframe, `client_id` is not an index because each id might appear more than once. The index for this dataframe is instead `loan_id`. 

When we create an `entity` in featuretools, we have to identify which column of the dataframe is the index. If the data does not have a unique index we can tell featuretools to make an index for the entity by passing in `make_index = True` and specifying a name for the index. If the data also has a uniquely identifying time index, we can pass that in as the `time_index` parameter. 

Featuretools will automatically infer the variable types (numeric, categorical, datetime) of the columns in our data, but we can also pass in specific datatypes to override this behavior. As an example, even though the `repaid` column in the `loans` dataframe is represented as an integer, we can tell featuretools that this is a categorical feature since it can only take on two discrete values. This is done using an integer with the variables as keys and the feature types as values.

In the code below we create the three entities and add them to the `EntitySet`.  The syntax is relatively straightforward with a few notes: for the `payments` dataframe we need to make an index, for the `loans` dataframe, we specify that `repaid` is a categorical variable, and for the `payments` dataframe, we specify that `missed` is a categorical feature. 

In [12]:
# Create an entity from the client dataframe
# This dataframe already has an index and a time index
es = es.entity_from_dataframe(entity_id = 'clients', dataframe = clients, 
                              index = 'client_id', time_index = 'joined')
es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 6]
  Relationships:
    No relationships

In [13]:
# Create an entity from the loans dataframe
# This dataframe already has an index and a time index
es = es.entity_from_dataframe(entity_id = 'loans', dataframe = loans, 
                              variable_types = {'repaid': ft.variable_types.Categorical},
                              index = 'loan_id', 
                              time_index = 'loan_start')
es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 6]
    loans [Rows: 443, Columns: 8]
  Relationships:
    No relationships

In [14]:
# Create an entity from the payments dataframe
# This does not yet have a unique index
es = es.entity_from_dataframe(entity_id = 'payments', 
                              dataframe = payments,
                              variable_types = {'missed': ft.variable_types.Categorical},
                              make_index = True,
                              index = 'payment_id',
                              time_index = 'payment_date')
es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 6]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    No relationships

All three entities have been successfully added to the `EntitySet`. We can access any of the entities using Python dictionary syntax.

In [15]:
es['clients']

Entity: clients
  Variables:
    client_id (dtype: index)
    joined (dtype: datetime_time_index)
    income (dtype: numeric)
    credit_score (dtype: numeric)
    join_month (dtype: numeric)
    log_income (dtype: numeric)
  Shape:
    (Rows: 25, Columns: 6)

In [16]:
es['loans']

Entity: loans
  Variables:
    loan_id (dtype: index)
    client_id (dtype: numeric)
    loan_type (dtype: categorical)
    loan_amount (dtype: numeric)
    loan_start (dtype: datetime_time_index)
    loan_end (dtype: datetime)
    rate (dtype: numeric)
    repaid (dtype: categorical)
  Shape:
    (Rows: 443, Columns: 8)

Featuretools correctly inferred each of the datatypes when we made this entity. We can also see that we overrode the type for the `repaid` feature, changing if from numeric to categorical. 

In [17]:
es['payments']

Entity: payments
  Variables:
    payment_id (dtype: index)
    loan_id (dtype: numeric)
    payment_amount (dtype: numeric)
    payment_date (dtype: datetime_time_index)
    missed (dtype: categorical)
  Shape:
    (Rows: 3456, Columns: 5)

## Relationships

After defining the entities (tables) in an `EntitySet`, we now need to tell featuretools [how they are related with a relationship](https://docs.featuretools.com/loading_data/using_entitysets.html#adding-a-relationship). The most intuitive way to think of relationships is with the parent to child analogy: a parent-to-child relationship is one-to-many because for each parent, there can be multiple children. The `client` dataframe is therefore the parent of the `loans` dataframe because while there is only one row for each client in the `client` dataframe, each client may have several previous loans covering multiple rows in the `loans` dataframe. Likewise, the `loans` dataframe is the parent of the `payments` dataframe because each loan will have multiple payments. 

These relationships are what allow us to group together datapoints using aggregation primitives and then create new features. As an example, we can group all of the previous loans associated with one client and find the average loan amount. We will discuss the features themselves more in a little bit, but for now let's define the relationships. 

To define relationships, we need to specify the parent variable and the child variable. This is the variable that links two entities together. In our example, the `client` and `loans` dataframes are linked together by the `client_id` column. Again, this is a parent to child relationship because for each `client_id` in the parent `client` dataframe, there may be multiple entries of the same `client_id` in the child `loans` dataframe. 

We codify relationships in the language of featuretools by specifying the parent variable and then the child variable. After creating a relationship, we add it to the `EntitySet`. 

In [18]:
# Relationship between clients and previous loans
r_client_previous = ft.Relationship(es['clients']['client_id'],
                                    es['loans']['client_id'])

# Add the relationship to the entity set
es = es.add_relationship(r_client_previous)

The relationship has now been stored in the entity set. The second relationship is between the `loans` and `payments`. These two entities are related by the `loan_id` variable.

In [19]:
# Relationship between previous loans and previous payments
r_payments = ft.Relationship(es['loans']['loan_id'],
                                      es['payments']['loan_id'])

# Add the relationship to the entity set
es = es.add_relationship(r_payments)

es

Entityset: clients
  Entities:
    clients [Rows: 25, Columns: 6]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    loans.client_id -> clients.client_id
    payments.loan_id -> loans.loan_id

We now have our entities in an entityset along with the relationships between them. We can now start to making new features from all of the tables using stacks of feature primitives to form deep features. First, let's cover feature primitives.


## Feature Primitives

A [feature primitive](https://docs.featuretools.com/automated_feature_engineering/primitives.html) a at a very high-level is an operation applied to data to create a feature. These represent very simple calculations that can be stacked on top of each other to create complex features. Feature primitives fall into two categories:

* __Aggregation__: function that groups together child datapoints for each parent and then calculates a statistic such as mean, min, max, or standard deviation. An example is calculating the maximum loan amount for each client. An aggregation works across multiple tables using relationships between tables.
* __Transformation__: an operation applied to one or more columns in a single table. An example would be extracting the day from dates, or finding the difference between two columns in one table.

Let's take a look at feature primitives in featuretools. We can view the list of primitives:

In [20]:
primitives = ft.list_primitives()
primitives.head()

,name,type,dask_compatible,koalas_compatible,description,valid_inputs,return_type
0,mean,aggregation,True,True,Computes the average for a list of values.,Numeric,Numeric
1,all,aggregation,True,False,Calculates if all values are 'True' in a list.,Boolean,Boolean
2,any,aggregation,True,False,Determines if any value is 'True' in a list.,Boolean,Boolean
3,count,aggregation,True,True,"Determines the total number of values, excludi...",Index,Numeric
4,max,aggregation,True,True,"Calculates the highest value, ignoring `NaN` v...",Numeric,Numeric


### Create custom primitives

In [21]:
# Simple Custom Primitives

# In [6]: from featuretools.primitives import make_agg_primitive, make_trans_primitive

# In [7]: from featuretools.variable_types import Text, Numeric

# In [8]: def absolute(column):
#    ...:     return abs(column)
#    ...: 

# In [9]: Absolute = make_trans_primitive(function=absolute,
#    ...:                                 input_types=[Numeric],
#    ...:                                 return_type=Numeric)

If featuretools does not have enough primitives for us, we can [also make our own.](https://docs.featuretools.com/automated_feature_engineering/primitives.html#defining-custom-primitives) 

To get an idea of what a feature primitive actually does, let's try out a few on our data. Using primitives is surprisingly easy using the `ft.dfs` function (which stands for deep feature synthesis). In this function, we specify the entityset to use; the `target_entity`, which is the dataframe we want to make the features for (where the features end up); the `agg_primitives` which are the aggregation feature primitives; and the `trans_primitives` which are the transformation primitives to apply. 

In the following example, we are using the `EntitySet` we already created, the target entity is the `clients` dataframe because we want to make new features about each client, and then we specify a few aggregation and transformation primitives. 

In [22]:
# ft.primitives.list_primitives().to_csv("list_prim.csv")

In [23]:
# Create new features using specified primitives
features, feature_names = ft.dfs(entityset = es, target_entity = 'clients', 
                                 agg_primitives = ['mean', 'max', 'percent_true', 'last'],
                                 trans_primitives = ['year', 'month', 'subtract_numeric', 'divide_numeric'])

In [24]:
features.head()

,income,credit_score,join_month,log_income,LAST(loans.loan_amount),LAST(loans.loan_id),LAST(loans.loan_type),LAST(loans.rate),LAST(loans.repaid),MAX(loans.loan_amount),...,join_month - MEAN(payments.payment_amount),log_income - MAX(loans.loan_amount),log_income - MAX(loans.rate),log_income - MAX(payments.payment_amount),log_income - MEAN(loans.loan_amount),log_income - MEAN(loans.rate),log_income - MEAN(payments.payment_amount),YEAR(LAST(loans.loan_end)),YEAR(LAST(loans.loan_start)),YEAR(LAST(payments.payment_date))
client_id,,,,,,,,,,,,,,,,,,,,,
42320,229481,563,4,12.343576,8090,10156,home,3.18,0,13887,...,-1017.483333,-13874.656424,5.603576,-2756.656424,-7049.723091,9.886242,-1009.139758,2015,2012,2013
39384,191204,617,6,12.161096,14654,11735,other,2.26,0,14654,...,-1187.630137,-14641.838904,2.931096,-2809.838904,-7853.312588,8.622675,-1181.469041,2016,2014,2015
26945,214516,806,11,12.276140,9249,11482,cash,2.86,1,14593,...,-1098.473214,-14580.723860,6.626140,-2755.723860,-7113.657194,9.420806,-1097.197075,2016,2013,2014
41472,152214,638,11,11.933043,10122,11936,cash,1.03,0,13657,...,-1118.076190,-13645.066957,2.113043,-2424.066957,-7498.879457,7.951793,-1117.143148,2016,2014,2015
46180,43851,562,11,10.688553,3834,10887,other,1.38,0,14081,...,-1175.550336,-14070.311447,1.428553,-2649.311447,-7690.161447,7.186053,-1175.861783,2016,2014,2015


In [25]:
#columns that are created
feature_names

[<Feature: income>,
 <Feature: credit_score>,
 <Feature: join_month>,
 <Feature: log_income>,
 <Feature: LAST(loans.loan_amount)>,
 <Feature: LAST(loans.loan_id)>,
 <Feature: LAST(loans.loan_type)>,
 <Feature: LAST(loans.rate)>,
 <Feature: LAST(loans.repaid)>,
 <Feature: MAX(loans.loan_amount)>,
 <Feature: MAX(loans.rate)>,
 <Feature: MEAN(loans.loan_amount)>,
 <Feature: MEAN(loans.rate)>,
 <Feature: LAST(payments.missed)>,
 <Feature: LAST(payments.payment_amount)>,
 <Feature: LAST(payments.payment_id)>,
 <Feature: MAX(payments.payment_amount)>,
 <Feature: MEAN(payments.payment_amount)>,
 <Feature: credit_score / income>,
 <Feature: credit_score / join_month>,
 <Feature: credit_score / log_income>,
 <Feature: income / credit_score>,
 <Feature: income / join_month>,
 <Feature: income / log_income>,
 <Feature: join_month / credit_score>,
 <Feature: join_month / income>,
 <Feature: join_month / log_income>,
 <Feature: log_income / credit_score>,
 <Feature: log_income / income>,
 <Feature:

Already we can see how useful featuretools is: it performed the same operations we did manually but also many more in addition. Examining the names of the features in the dataframe brings us to the final piece of the puzzle: deep features.

## Deep Feature Synthesis

While feature primitives are useful by themselves, the main benefit of using featuretools arises when we stack primitives to get deep features. The depth of a feature is simply the number of primitives required to make a feature. So, a feature that relies on a single aggregation would be a deep feature with a depth of 1, a feature that stacks two primitives would have a depth of 2 and so on. The idea itself is lot simpler than the name "deep feature synthesis" implies. (I think the authors were trying to ride the way of deep neural network hype when they named the method!) To read more about deep feature synthesis, check out [the documentation](https://docs.featuretools.com/automated_feature_engineering/afe.html) or the [original paper by Max Kanter et al](http://www.jmaxkanter.com/static/papers/DSAA_DSM_2015.pdf). 

Already in the dataframe we made by specifying the primitives manually we can see the idea of feature depth. For instance, the MEAN(loans.loan_amount) feature has a depth of 1 because it is made by applying a single aggregation primitive. This feature represents the average size of a client's previous loans.

As well scroll through the features, we see a number of features with a depth of 2. For example, the LAST(loans.(MEAN(payments.payment_amount))) has depth = 2 because it is made by stacking two feature primitives, first an aggregation and then a transformation. This feature represents the average payment amount for the last (most recent) loan for each client.

We can create features of arbitrary depth by stacking more primitives. However, when I have used featuretools I've never gone beyond a depth of 2! After this point, the features become very convoluted to understand. I'd encourage anyone interested to experiment with increasing the depth (maybe for a real problem) and see if there is value to "going deeper".

## Automated Deep Feature Synthesis

In addition to manually specifying aggregation and transformation feature primitives, we can let featuretools automatically generate many new features. We do this by making the same `ft.dfs` function call, but without passing in any primitives. We just set the `max_depth` parameter and featuretools will automatically try many all combinations of feature primitives to the ordered depth. 

When running on large datasets, this process can take quite a while, but for our example data, it will be relatively quick. For this call, we only need to specify the `entityset`, the `target_entity` (which will again be `clients`), and the `max_depth`. 

In [28]:
# Perform deep feature synthesis without specifying primitives
features, feature_names = ft.dfs(entityset=es, target_entity='clients', 
                                 max_depth = 2,verbose = 1)

Built 112 features
Elapsed: 00:00 | Progress: 100%|███████████████████████████████████████████████████████████████████████████████████████


In [29]:
features.head()

,income,credit_score,join_month,log_income,COUNT(loans),MAX(loans.loan_amount),MAX(loans.rate),MEAN(loans.loan_amount),MEAN(loans.rate),MIN(loans.loan_amount),...,MODE(payments.loans.repaid),NUM_UNIQUE(payments.loans.client_id),NUM_UNIQUE(payments.loans.loan_type),NUM_UNIQUE(payments.loans.repaid),SKEW(payments.loans.loan_amount),SKEW(payments.loans.rate),STD(payments.loans.loan_amount),STD(payments.loans.rate),SUM(payments.loans.loan_amount),SUM(payments.loans.rate)
client_id,,,,,,,,,,,,,,,,,,,,,
42320,229481,563,4,12.343576,15,13887,6.74,7062.066667,2.457333,1070,...,1,1,4,2,0.097213,0.872916,3929.351652,1.891948,843367,302.84
39384,191204,617,6,12.161096,19,14654,9.23,7865.473684,3.538421,1770,...,1,1,4,2,-0.277360,0.971757,3765.368569,2.685701,1161741,499.12
26945,214516,806,11,12.276140,15,14593,5.65,7125.933333,2.855333,653,...,0,1,4,2,0.289824,0.020986,4372.558827,1.599556,771053,330.13
41472,152214,638,11,11.933043,16,13657,9.82,7510.812500,3.981250,986,...,1,1,4,2,-0.064213,0.311596,4250.127784,3.129716,784731,435.36
46180,43851,562,11,10.688553,20,14081,9.26,7700.850000,3.502500,1607,...,0,1,4,2,0.127445,0.649145,3815.702946,2.714351,1142666,578.43


### Tuning DFS using deep features as per business needs using seed values

In [30]:
high_income= ft.Feature(es["loans"]["loan_amount"]) > 50000
features, feature_names = ft.dfs(entityset=es, target_entity='clients', 
                                 max_depth = 2,verbose = 1, seed_features=[high_income])

Built 114 features
Elapsed: 00:00 | Progress: 100%|███████████████████████████████████████████████████████████████████████████████████████


In [31]:
features.head()

,income,credit_score,join_month,log_income,COUNT(loans),MAX(loans.loan_amount),MAX(loans.rate),MEAN(loans.loan_amount),MEAN(loans.rate),MIN(loans.loan_amount),...,NUM_UNIQUE(payments.loans.client_id),NUM_UNIQUE(payments.loans.loan_type),NUM_UNIQUE(payments.loans.repaid),SKEW(payments.loans.loan_amount),SKEW(payments.loans.rate),STD(payments.loans.loan_amount),STD(payments.loans.rate),SUM(payments.loans.loan_amount),SUM(payments.loans.rate),PERCENT_TRUE(payments.loans.loan_amount > 50000)
client_id,,,,,,,,,,,,,,,,,,,,,
42320,229481,563,4,12.343576,15,13887,6.74,7062.066667,2.457333,1070,...,1,4,2,0.097213,0.872916,3929.351652,1.891948,843367,302.84,0.0
39384,191204,617,6,12.161096,19,14654,9.23,7865.473684,3.538421,1770,...,1,4,2,-0.277360,0.971757,3765.368569,2.685701,1161741,499.12,0.0
26945,214516,806,11,12.276140,15,14593,5.65,7125.933333,2.855333,653,...,1,4,2,0.289824,0.020986,4372.558827,1.599556,771053,330.13,0.0
41472,152214,638,11,11.933043,16,13657,9.82,7510.812500,3.981250,986,...,1,4,2,-0.064213,0.311596,4250.127784,3.129716,784731,435.36,0.0
46180,43851,562,11,10.688553,20,14081,9.26,7700.850000,3.502500,1607,...,1,4,2,0.127445,0.649145,3815.702946,2.714351,1142666,578.43,0.0


In [32]:
feature_names

[<Feature: income>,
 <Feature: credit_score>,
 <Feature: join_month>,
 <Feature: log_income>,
 <Feature: COUNT(loans)>,
 <Feature: MAX(loans.loan_amount)>,
 <Feature: MAX(loans.rate)>,
 <Feature: MEAN(loans.loan_amount)>,
 <Feature: MEAN(loans.rate)>,
 <Feature: MIN(loans.loan_amount)>,
 <Feature: MIN(loans.rate)>,
 <Feature: MODE(loans.loan_type)>,
 <Feature: MODE(loans.repaid)>,
 <Feature: NUM_UNIQUE(loans.loan_type)>,
 <Feature: NUM_UNIQUE(loans.repaid)>,
 <Feature: SKEW(loans.loan_amount)>,
 <Feature: SKEW(loans.rate)>,
 <Feature: STD(loans.loan_amount)>,
 <Feature: STD(loans.rate)>,
 <Feature: SUM(loans.loan_amount)>,
 <Feature: SUM(loans.rate)>,
 <Feature: COUNT(payments)>,
 <Feature: MAX(payments.payment_amount)>,
 <Feature: MEAN(payments.payment_amount)>,
 <Feature: MIN(payments.payment_amount)>,
 <Feature: MODE(payments.missed)>,
 <Feature: NUM_UNIQUE(payments.missed)>,
 <Feature: SKEW(payments.payment_amount)>,
 <Feature: STD(payments.payment_amount)>,
 <Feature: SUM(payments

### Encoding categorical features

In [33]:
feature_matrix, feature_defs = ft.dfs(entityset=es, target_entity='clients', verbose = 1,agg_primitives=["mode"],max_depth=1)
feature_matrix_enc, features_enc = ft.encode_features(feature_matrix, feature_defs)
feature_matrix_enc.head()

Built 10 features
Elapsed: 00:00 | Progress: 100%|███████████████████████████████████████████████████████████████████████████████████████


,income,credit_score,join_month,log_income,MODE(loans.loan_type) = home,MODE(loans.loan_type) = cash,MODE(loans.loan_type) = credit,MODE(loans.loan_type) = other,MODE(loans.loan_type) is unknown,MODE(loans.repaid) = 1,...,YEAR(joined) = 2002,YEAR(joined) = 2000,YEAR(joined) = 2011,YEAR(joined) = 2009,YEAR(joined) = 2006,YEAR(joined) = 2001,YEAR(joined) = 2013,YEAR(joined) = 2012,YEAR(joined) = 2010,YEAR(joined) is unknown
client_id,,,,,,,,,,,,,,,,,,,,,
42320,229481,563,4,12.343576,True,False,False,False,False,True,...,False,True,False,False,False,False,False,False,False,False
39384,191204,617,6,12.161096,False,False,True,False,False,True,...,False,True,False,False,False,False,False,False,False,False
26945,214516,806,11,12.276140,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,False,False
41472,152214,638,11,11.933043,False,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
46180,43851,562,11,10.688553,False,False,False,True,False,False,...,False,False,False,False,False,True,False,False,False,False


Deep feature synthesis has created 90 new features out of the existing data! While we could have created all of these manually, I am glad to not have to write all that code by hand. The primary benefit of featuretools is that it creates features without any subjective human biases. Even a human with considerable domain knowledge will be limited by their imagination when making new features (not to mention time). Automated feature engineering is not limited by these factors (instead it's limited by computation time) and provides a good starting point for feature creation. This process likely will not remove the human contribution to feature engineering completely because a human can still use domain knowledge and machine learning expertise to select the most important features or build new features from those suggested by automated deep feature synthesis.

# Next Steps

While automatic feature engineering solves one problem, it provides us with another problem: too many features! Although it's difficult to say which features will be important to a given machine learning task ahead of time, it's likely that not all of the features made by featuretools add value. In fact, having too many features is a significant issue in machine learning because it makes training a model much harder. The [irrelevant features can drown out the important features](https://pdfs.semanticscholar.org/a83b/ddb34618cc68f1014ca12eef7f537825d104.pdf), leaving a model unable to learn how to map the features to the target.

This problem is known as the ["curse of dimensionality"](https://en.wikipedia.org/wiki/Curse_of_dimensionality#Machine_learning) and is addressed through the process of [feature reduction and selection](http://scikit-learn.org/stable/modules/feature_selection.html), which means [removing low-value features](https://machinelearningmastery.com/feature-selection-machine-learning-python/) from the data. Defining which features are useful is an important problem where a data scientist can still add considerable value to the feature engineering task. Feature reduction will have to be another topic for another day!

# Conclusions

In this notebook, we saw how to apply automated feature engineering to an example dataset. This is a powerful method which allows us to overcome the human limits of time and imagination to create many new features from multiple tables of data. Featuretools is built on the idea of deep feature synthesis, which means stacking multiple simple feature primitives - __aggregations and transformations__ - to create new features. Feature engineering allows us to combine information across many tables into a single dataframe that we can then use for machine learning model training. Finally, the next step after creating all of these features is figuring out which ones are important. 

Featuretools is currently the only Python option for this process, but with the recent emphasis on automating aspects of the machine learning pipeline, other competitiors will probably enter the sphere. While the exact tools will change, the idea of automatically creating new features out of existing data will grow in importance. Staying up-to-date on methods such as automated feature engineering is crucial in the rapidly changing field of data science. Now go out there and find a problem on which to apply featuretools! 

For more information, check out the [documentation for featuretools](https://docs.featuretools.com/index.html). Also, read about how featuretools is [used in the real world by Feature Labs](https://www.featurelabs.com/), the company behind the open-source library.